In [115]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [116]:
car_dataframe = pd.read_csv("data/car details v4.csv")
data = (car_dataframe.loc[:, ["Fuel Tank Capacity", "Year", "Price"]]).to_numpy().astype('float64')
data_clean = data[~np.isnan(data).any(axis=1)]
print(data_clean)
x = data_clean[:, :2]
y = data_clean[:, 2:]
x = x / x.mean()
y = y / y.mean()
print(x)
print(y)

[[3.500e+01 2.017e+03 5.050e+05]
 [4.200e+01 2.014e+03 4.500e+05]
 [3.500e+01 2.011e+03 2.200e+05]
 ...
 [3.200e+01 2.014e+03 2.750e+05]
 [4.500e+01 2.013e+03 2.400e+05]
 [6.500e+01 2.018e+03 4.290e+06]]
[[0.03383984 1.95014137]
 [0.0406078  1.94724081]
 [0.03383984 1.94434025]
 ...
 [0.03093928 1.94724081]
 [0.04350836 1.94627396]
 [0.06284541 1.95110822]]
[[0.30088322]
 [0.26811376]
 [0.13107784]
 ...
 [0.1638473 ]
 [0.14299401]
 [2.55601784]]


In [117]:
def make_pred(x, w, b):
    return np.matmul(x, w) + b

def compute_cost(x, y, w, b):
    sqr_diff = (make_pred(x, w, b) - y)**2
    return sqr_diff.sum() / (2*x.shape[1])

def run_grad_decent(x, y, w, b, alpha):
    cost_sum = ((make_pred(x, w, b)-y).sum() / x.shape[1])
    new_b = b - alpha * cost_sum

    j = x.shape[1]
    i = x.shape[0]
    new_w = np.empty(j)
    for k in range(i):
        pred = w.reshape(-1).dot(x[k]) + b - y[k]
        for l in range(j):
            new_w[l] += (pred*x[k][l]).item()
    new_w /= j
    new_w = new_w.reshape([j, 1])
    
    return new_w, new_b

def run_linear_regression(x, y, alpha = .0001, iter = 5):
    j = x.shape[1]
    w = np.empty([j, 1])
    b = 0
    print(compute_cost(x, y, w, b))
    for _ in range(iter):
        w, b = run_grad_decent(x, y, w, b, alpha)
        print(compute_cost(x, y, w, b))
    return w, b

In [118]:
run_linear_regression(x, y)

1476.3012173916443
5.881736885502571e+37
8.057095780536327e+44
1.103701125997567e+52
1.511904796351913e+59
2.0710825235100486e+66


(array([[-8.62486929e+29],
        [-3.34431210e+31]]),
 1.7157358181630234e+27)